In [34]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.feature import blob_log

# Define the path to the folder containing the images for RGB channel distribution
image_folder_path = 'Dataset/clean/train/infected_processed/'

# Choose one image filename for analysis 
image_filename = 'C100P61ThinF_IMG_20150918_144104_cell_162.png'

# Construct the full path to the chosen image
image_path = os.path.join(image_folder_path, image_filename)

# Load the selected image
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Function to calculate blob features from an RGB image
def calculate_blob_features(image):
    # Detect blobs using the Laplacian of Gaussian (LoG) method on the RGB image
    blobs = blob_log(image, max_sigma=30, threshold=0.02)
    
    # If no blobs are found, return zeros
    if len(blobs) == 0:
        return [0, 0]
    
    # Extract blob sizes (radii)
    blob_sizes = blobs[:, 2]
    
    # Calculate mean and max blob sizes
    mean_blob_size = np.mean(blob_sizes)
    max_blob_size = np.max(blob_sizes)
    
    return [mean_blob_size, max_blob_size]


# Function to calculate edge density from an image
def calculate_edge_density(image):
    # Convert the image to grayscale
    image_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    # Detect edges using the Canny edge detector
    edges = cv2.Canny(image_gray, threshold1=30, threshold2=100)
    
    # Calculate edge density
    total_pixels = image_gray.size
    edge_pixels = np.count_nonzero(edges)
    edge_density = edge_pixels / total_pixels
    
    return edge_density

# Function to calculate average RGB values from an image
def calculate_average_rgb(image):
    red_mean = np.mean(image[:, :, 0])
    green_mean = np.mean(image[:, :, 1])
    blue_mean = np.mean(image[:, :, 2])
    return red_mean, green_mean, blue_mean

# Calculate blob features, edge density, and average RGB values for the selected image
blob_features = calculate_blob_features(image_rgb)
edge_density = calculate_edge_density(image_rgb)
red_mean, green_mean, blue_mean = calculate_average_rgb(image_rgb)

# Create a DataFrame to store the analysis results
data = {
    'ImageFileName': [image_filename],
    'MeanBlobSize': [blob_features[0]],
    'MaxBlobSize': [blob_features[1]],
    'EdgeDensity': [edge_density],
    'RedChannelMean': [red_mean],
    'GreenChannelMean': [green_mean],
    'BlueChannelMean': [blue_mean]
}

analysis_df = pd.DataFrame(data)

analysis_df.head()



,ImageFileName,MeanBlobSize,MaxBlobSize,EdgeDensity,RedChannelMean,GreenChannelMean,BlueChannelMean
0,C100P61ThinF_IMG_20150918_144104_cell_162.png,0.666667,2.0,0.1744,163.5904,119.1104,119.3664
